## Data Cleaning and Preparation
***

In [ ]:
import pandas as pd
import numpy as np
import glob
import re
from bs4 import BeautifulSoup 
import requests
import pickle 
import matplotlib.pyplot as plt

In [ ]:
# Load in T100 data from csv files

files_li = glob.glob('zippedData/T_T100D*.2.csv') 
bts_df = pd.DataFrame(pd.read_csv(files_li[0])) 
for i in range(1,len(files_li)): 
    data = pd.read_csv(files_li[i]) 
    df1 = pd.DataFrame(data) 
    bts_df = pd.concat([df1,bts_df],axis=0, ignore_index=True) 

In [ ]:
# Keep only top 40 used aircraft models

bts_40 = bts_df['AIRCRAFT_TYPE'].value_counts().head(40).index.to_list()
indexNum = bts_df[~bts_df['AIRCRAFT_TYPE'].isin(bts_40)].index
bts_df.drop(indexNum , inplace=True)

In [ ]:
# Get list of top 40 aircraft names for data cleaning

#bts_des = bts_df['AIRCRAFT_TYPE'].value_counts.to_frame()
bts_des = pd.DataFrame(pd.unique(bts_df['AIRCRAFT_TYPE']), columns =['AIRCRAFT_TYPE'])

In [ ]:
# BTS uses number code to identify aircraft type in it's flight data. The legend is in a seperate csv file. 
# Load in the legend for aircraft type. 

ac_df = pd.read_csv('zippedData/L_AIRCRAFT_TYPE.csv')

In [ ]:
# Merge flight data with aircfaft type. This will match BTS type names to the fligt data.

bts_des = pd.merge(bts_des,ac_df,left_on='AIRCRAFT_TYPE',right_on='Code')

In [ ]:
# Drop piston aircraft
#bts_des = bts_des.drop(index=[5, 10, 18, 26, 29, 31, 37])
i = bts_des[(bts_des['Code'] == 40)].index
j = bts_des[(bts_des['Code'] == 35)].index
k = bts_des[(bts_des['Code'] == 194)].index
l = bts_des[(bts_des['Code'] == 79)].index
m = bts_des[(bts_des['Code'] == 479)].index
n = bts_des[(bts_des['Code'] == 415)].index
o = bts_des[(bts_des['Code'] == 416)].index
bts_des = bts_des.drop(i)
bts_des = bts_des.drop(j)
bts_des = bts_des.drop(k)
bts_des = bts_des.drop(l)
bts_des = bts_des.drop(m)
bts_des = bts_des.drop(n)
bts_des = bts_des.drop(o)


In [ ]:
common_words = ['BOEING','CHEROKEE', 'AIRBUS', 'INDUSTRIE', 'CANADAIR', 'EMBRAER', 'MCDONNELL', 'DOUGLAS', 
                'DE', 'HAVILLAND', 'BEAVER', 'SUPER', 'NAVAJO', 'EM', 'ER', 'BEECH']

In [ ]:
common_words_eng = ['ILYUSHIN', 'ANTONOV', 'JUNKERS', 'LOCKHEED', 'BOEING','AIRBUS', 'INDUSTRIE', 
                    'CANADAIR', 'EMBRAER', 'STATIONAIR','MCDONNELL', 'DOUGLAS', 'DE', 'HAVILLAND', 'BEAVER',
                    'SUPER','NAVAJO', 'EM', 'ER', 'BEECH', 'CRAFT', 'BOMBARDIER','CANADA']

In [ ]:
common_words_emi = ['TRENT', 'CFMI', 'PRATT & WHITNEY', 'GENERAL ELECTRIC', 'CANADA',
                    'ROLLSROYCE', 'LYCOMING', 'ALLISON', 'GARRETT', 'CONTINENTAL', 'IAE']

In [ ]:
def short_name(name, x):
    if len(name) >= x:
        short_name = name[:x]
        return(short_name)
    return(name)

In [ ]:
def remove_dash(name):
    name = name.replace('-', '')
    return(name)

In [ ]:
def convert_upper(name):
    name = name.upper()
    return(name)

In [ ]:
def convert_date(date):
    date = date.astype('datetime64[ns]')
    return(date)

In [ ]:
def remove_slash(name):
    res = re.match(r"^[^/]*", name)
    return(str(res.group())) 

In [ ]:
def remove_common(name, common_w):
    for common in common_w:
        name = name.replace(common, '')
    return(name)

In [ ]:
def remove_space(name):
    name = name.replace(' ', "")
    return(name)

In [ ]:

def convert_boeing(name):
    if re.match(r"^(7)(.)(7)", name):
        name = name[:3]
    return name

In [ ]:
def convert_crj(name):
    if re.match(r"^RJ", name):
        name = name.replace('RJ', 'CRJ')
    return name

In [ ]:
bts_des.loc[bts_des['Code'] == 629, 'Description'] = 'Canadair RJ-200'
bts_des.loc[bts_des['Code'] == 631, 'Description'] = 'Canadair RJ-700'
bts_des.loc[bts_des['Code'] == 638, 'Description'] = 'Canadair RJ-900'
bts_des.loc[bts_des['Code'] == 838, 'Description'] = 'Boeing 737-800'
bts_des.loc[bts_des['Code'] == 674, 'Description'] = 'Embraer ERJ-130'
bts_des.loc[bts_des['Code'] == 676, 'Description'] = 'Embraer ERJ-140'
bts_des.loc[bts_des['Code'] == 675, 'Description'] = 'Embraer ERJ-145'
bts_des.loc[bts_des['Code'] == 677, 'Description'] = 'Embraer ERJ-170'
bts_des.loc[bts_des['Code'] == 678, 'Description'] = 'Embraer ERJ-190'

In [ ]:
bts_des['Description'] = [convert_upper(x) for x in bts_des['Description']]
bts_des['Description'] = [remove_slash(x) for x in bts_des['Description']]
bts_des['Description'] = [remove_dash(x) for x in bts_des['Description']]
bts_des['Description'] = [remove_common(x, common_words) for x in bts_des['Description']]
bts_des['Description'] = [remove_space(x) for x in bts_des['Description']]
bts_des['Description'] = [short_name(x, 4) for x in bts_des['Description']]
bts_des['Description'] = [convert_boeing(x) for x in bts_des['Description']]
bts_des['Description'] = [convert_crj(x) for x in bts_des['Description']]

In [ ]:
bts_df = pd.merge(bts_df,bts_des[['Code', 'Description']],left_on='AIRCRAFT_TYPE', right_on='Code')

In [ ]:
bts_des.drop(['AIRCRAFT_TYPE', 'Code'], axis=1, inplace=True)

In [ ]:
# Import Data Set from CSV

crash_df = pd.read_csv('zippedData/AviationData.csv', encoding='latin-1', dtype={6: str, 7: str, 14: str, 15: str,
                                                                                 28: str})

In [ ]:
crash_df.columns

In [ ]:
# Drop rows before 2012 so we can match data by year in bts_df
date_mask = (crash_df['Event.Date'] > '2011-12-31')

# THIS RETURNS ALL ROWS GREATER THAN THE DATE PROVIDED ABOVE
crash_df = crash_df.loc[date_mask]

# Rows without an aircraft model type don't help use. Dropping these rows.
crash_df = crash_df.dropna(subset=['Model'])

In [ ]:
# Some values in Make are upper, and some are mixed case. Change the Make column to all uppercase
#crash_df['Make'] = crash_df['Make'].str.upper()

# Convert Event.Date from string to datetime. Used to filter df on years
crash_df['Event.Date'] = crash_df['Event.Date'].astype('datetime64[ns]')

In [ ]:
crash_df['Model'] = [convert_upper(x) for x in crash_df['Model']]
#crash_df['Event.Date'] = [convert_date(x) for x in crash_df['Event.Date']]
crash_df['Model'] = [remove_slash(x) for x in crash_df['Model']]
crash_df['Model'] = [remove_dash(x) for x in crash_df['Model']]
crash_df['Model'] = [remove_common(x, common_words) for x in crash_df['Model']]
crash_df['Model'] = [remove_space(x) for x in crash_df['Model']]
crash_df['Model'] = [short_name(x, 4) for x in crash_df['Model']]
crash_df['Model'] = [convert_boeing(x) for x in crash_df['Model']]
#crash_df['Model'].replace('PA-', 'PA', inplace=True, regex=True)

In [ ]:
def match_types(model):
    for newmodel in bts_des['Description']:
        if newmodel.startswith(model):
            #print(f'Processing: {model} \t:{newmodel}')
            return newmodel

In [ ]:
crash_df['NewModel'] = [match_types(model) for model in crash_df['Model']]

## Engine Data

In [ ]:
# In place of running this and making a web request, import the pickle file from below
# Assign URL |
#url = "https://asn.flightsafety.org/database/engines/"

# Make a GET request to fetch the raw HTML content 
#html_content = requests.get(url).text 

# Parse the response with html.parser
#soup = BeautifulSoup(html_content,"html.parser") 

# The data we need are wrapped in <a> tags. Grab all the <a> tags here and we'll filter later.
#datas = soup.find_all("a")


#Make list of URLs and list oF Aircraft to create dictionary
# k = []
# v = []
# for item in datas:
#     if 'engine' in item['href']:
#         k.append(item['href'])
#         v.append(item.text)

# lookup_dict = {'URL': k, 'Engine': v}


#url_df = pd.DataFrame.from_dict(lookup_dict)

# with open('lookup_dict.pkl', 'wb') as f:
#     pickle.dump(lookup_dict, f)

with open('zippedData/lookup_dict.pkl', 'rb') as f:
    lookup_dict = pickle.load(f)

In [ ]:
# Take in the engine and retriev the matching data page with Aircraft data
# Add Aircraft list from web a python list.
# Return the python list

# url_p1 = 'https://asn.flightsafety.org'
# def lookup_ac(url_p2):
#     return_list = []
#     url = f'{url_p1}{url_p2}'
#     sub_content = requests.get(url).text
#     soup2 = BeautifulSoup(sub_content,"html.parser")
#     u_list=soup2.find_all('ul')[1]
#     i = 0 
#     while i < len(u_list.select('li')):
#         (u_list.select('li')[i].text)
#         return_list.append(u_list.select('li')[i].text)
#         i += 1
#     return(return_list)

# Create Aircraft column to list aircraft for each engine.
#url_df['Aircraft'] = url_df['URL'].map(lookup_ac)

# Save df to pickle file.
#url_df.to_pickle('zippedData/url_df.pkl')

with open('zippedData/url_df.pkl', 'rb') as f:
    url_df = pickle.load(f)

In [ ]:
# Create a row for each aircraft for each engine. 
# Iterate through url_df['Aircraft'] column. For each element
# in list, create a new row.

engines = []
aircraft = []
for idx, row in url_df.iterrows():
    for ac in row['Aircraft']:
        engines.append(row['Engine'])
        aircraft.append(ac)

In [ ]:
# Create dictionary from lists to load into dataFrame
ac_eng_dict = {'Engine': engines, 'Aircraft': aircraft}

In [ ]:
# Create dataFrame from dictionary
engines_df = pd.DataFrame.from_dict(ac_eng_dict)

In [ ]:
# df.loc[len(df.index)] = ['Amy', 89, 93] 
engines_df.loc[engines_df['Aircraft'] == 'Bombardier CRJ100 / 200 / 440'] = ['General Electric CF34','Canadair RJ-100']
engines_df.loc[engines_df['Aircraft'] == 'Bombardier CRJ700'] = ['General Electric CF34','Canadair RJ-700']
engines_df.loc[engines_df['Aircraft'] == 'Bombardier CRJ900'] = ['General Electric CF34','Canadair RJ-900']
engines_df.loc[len(engines_df.index)] = ['General Electric CF34', 'Canadair RJ-200']
engines_df.loc[len(engines_df.index)] = ['General Electric CF34', 'Canadair RJ-440']
engines_df.loc[len(engines_df.index)] = ['General Electric CF34', 'Embraer ERJ-130']
engines_df.loc[len(engines_df.index)] = ['General Electric CF34', 'Embraer ERJ-135']
engines_df.loc[len(engines_df.index)] = ['General Electric CF34', 'Embraer ERJ-140']
engines_df.loc[len(engines_df.index)] = ['General Electric CF34', 'Embraer ERJ-175']
engines_df.loc[len(engines_df.index)] = ['General Electric CF34', 'Embraer ERJ-190']
engines_df.loc[len(engines_df.index)] = ['Pratt & Whitney JT8D', 'McDonnell Douglas DC-9-80']

In [ ]:
engines_df['Aircraft'] = [convert_upper(x) for x in engines_df['Aircraft']]
engines_df['Aircraft'] = [remove_slash(x) for x in engines_df['Aircraft']]
engines_df['Aircraft'] = [remove_dash(x) for x in engines_df['Aircraft']]
engines_df['Aircraft'] = [remove_common(x, common_words_eng) for x in engines_df['Aircraft']]
engines_df['Aircraft'] = [remove_space(x) for x in engines_df['Aircraft']]
engines_df['Aircraft'] = [short_name(x, 4) for x in engines_df['Aircraft']]
engines_df['Aircraft'] = [convert_boeing(x) for x in engines_df['Aircraft']]
engines_df['Aircraft'] = [convert_crj(x) for x in engines_df['Aircraft']]

In [ ]:
engines_df['Engine_Name'] = [convert_upper(x) for x in engines_df['Engine']]
engines_df['Engine_Name'] = [remove_slash(x) for x in engines_df['Engine_Name']]
engines_df['Engine_Name'] = [remove_dash(x) for x in engines_df['Engine_Name']]
engines_df['Engine_Name'] = [remove_common(x, common_words_emi) for x in engines_df['Engine_Name']]
engines_df['Engine_Name'] = [remove_common(x, common_words_emi) for x in engines_df['Engine_Name']]
engines_df['Engine_Name'] = [remove_space(x) for x in engines_df['Engine_Name']]
engines_df['Engine_Name'] = [short_name(x, 5) for x in engines_df['Engine_Name']]
#engines_df['Engine_Name'] = [convert_boeing(x) for x in engines_df['Engine_Name']]
#engines_df['Engine_Name'] = [convert_crj(x) for x in engines_df['Engine_Name']]

In [ ]:
bts_des = pd.merge(bts_des, engines_df,left_on='Description', right_on='Aircraft', how='left')

In [ ]:
bts_des.drop_duplicates(inplace=True)

## Emissions Data

In [ ]:
emissions_df = pd.read_excel('zippedData/edb-emissions-databank_v30__web_.xlsx',
                             sheet_name='Gaseous Emissions and Smoke', 
                             usecols=('A:D, AJ'))

In [ ]:
emissions_df['Engine Identification'] = [convert_upper(x) for x in emissions_df['Engine Identification']]
emissions_df['Engine Identification'] = [remove_slash(x) for x in emissions_df['Engine Identification']]
emissions_df['Engine Identification'] = [remove_dash(x) for x in emissions_df['Engine Identification']]
emissions_df['Engine Identification'] = [remove_common(x, common_words_emi) for x in emissions_df['Engine Identification']]
emissions_df['Engine Identification'] = [remove_space(x) for x in emissions_df['Engine Identification']]
emissions_df['Engine Identification'] = [short_name(x, 5) for x in emissions_df['Engine Identification']]

In [ ]:
emissions_df.at[527,'Engine Identification'] = 'PW100'
emissions_df.at[611,'Engine Identification'] = 'PW200'
emissions_df.at[633,'Engine Identification'] = 'PW400'

In [ ]:
def match_engine(str1):
    tmp_df = pd.DataFrame()
    mask = tmp_df['engine_id_starts_with_CFM'] = list( 
        map(lambda x: x.startswith(str1), emissions_df['Engine Identification'])) 
    filtered_df = emissions_df[mask]
    #return(filtered_df.at[0,1]['CO Dp/Foo Avg (g/kN)']).head(1)
    if filtered_df.shape[0] < 1:
        return(0)
    return(filtered_df.iat[0,4])

In [ ]:
bts_des['CO2'] = [match_engine(x) for x in bts_des['Engine_Name']]

In [ ]:
def get_best_eng(aircraft):
    ac_ser = bts_des.loc[bts_des['Aircraft'] == aircraft]
    return(ac_ser.sort_values('CO2')[:1])

## Data Processing

In [ ]:
# Some rows have NaN for values. Replace these with 0 value. 
crash_df['Total.Fatal.Injuries'] = crash_df['Total.Fatal.Injuries'].fillna(0)
crash_df['Total.Serious.Injuries'] = crash_df['Total.Serious.Injuries'].fillna(0)
crash_df['Total.Minor.Injuries'] = crash_df['Total.Minor.Injuries'].fillna(0)

In [ ]:
# Our scoring system will use 3 points for fatality; 1 point for serious injury and .5 points for minor injury.
crash_df['Injury_Score'] = crash_df['Total.Fatal.Injuries'].map(lambda x: x*3)
crash_df['Injury_Score'] = crash_df['Injury_Score'] + crash_df['Total.Serious.Injuries']
crash_df['Injury_Score'] = crash_df['Injury_Score'] + (crash_df['Total.Minor.Injuries'] * .5)

In [ ]:
total_score = crash_df.groupby('NewModel')['Injury_Score'].sum().to_frame()

In [ ]:
total_score['Count'] = bts_df.groupby('Description').size()

In [ ]:
total_score['Rating'] = (total_score['Injury_Score'] / total_score['Count']) * 1000

In [ ]:
total_score = pd.merge(total_score, bts_des[['CO2', 'Aircraft', 'Engine']], left_on='NewModel', right_on='Aircraft', how='left')

In [ ]:
total_score.columns

In [ ]:
total_score.sort_values(['CO2', 'Rating'], ascending=[True, True])

In [ ]:
plt.scatter(total_score['CO2'], total_score['Rating'])
plt.show()

## Airport Data Analysis


In [ ]:
ap_df = pd.read_csv('zippedData/us-airports.csv')
cbsa_df = pd.read_csv('zippedData/cbsa-est2023-alldata.csv', encoding='latin-1')

In [ ]:
ap_df = ap_df.dropna(subset=['gps_code'])
ap_df.drop(['continent', 'country_name', 'home_link', 'wikipedia_link', 'last_updated'], axis=1, inplace=True)

In [ ]:
ap_df = ap_df[(ap_df['type'] == 'medium_airport') | (ap_df['type'] == 'large_airport')]

In [ ]:
cbsa_df.drop(cbsa_df[cbsa_df['POPESTIMATE2023'] <= 550000].index, inplace = True)

In [ ]:
def get_state(str1):
    res = str1.split(',')
    return(res)
cbsa_df['STATE'] = [(get_state(x))[1].strip() for x in cbsa_df['NAME']]
cbsa_df['CITY'] = [(get_state(x))[0].split('-')[0] for x in cbsa_df['NAME']]

In [ ]:
result2 = ap_df.groupby(['municipality', 'local_region'])

In [ ]:
result2.get_group(('Cleveland','OH'))

In [ ]:
# extract keys from groups
keys = result2.groups.keys()
 
for i in keys:
    print(result2.get_group(i))
    print('\n')